# Multimodal Hate Speech Detection - DATASET



This notebook contains data analysis and testing for the Arabic multimodal hate speech detection task.

## 1. Download Data

Download and extract the Prop2Hate-Meme dataset:


In [ ]:
from datasets import load_dataset

dataset = load_dataset("QCRI/Prop2Hate-Meme")

# Specify the directory where you want to save the dataset

output_dir="./Prop2Hate-Meme"

# Save the dataset to the specified directory. This will save all splits to the output directory.
dataset.save_to_disk(output_dir)

# If you want to get the raw images from HF dataset format

from PIL import Image
import os
import json

# Directory to save the images
output_dir="./Prop2Hate-Meme/"
os.makedirs(output_dir, exist_ok=True)

# Iterate over the dataset and save each image
for split in ['train','dev','test']:     
    jsonl_path = os.path.join(output_dir, f"arabic_hateful_meme_{split}.jsonl")
    with open(jsonl_path, 'w', encoding='utf-8') as f:
        for idx, item in enumerate(dataset[split]):
            # Access the image directly as it's already a PIL.Image object
            image = item['image']
            image_path = os.path.join(output_dir, item['img_path'])
            # Ensure the directory exists
            os.makedirs(os.path.dirname(image_path), exist_ok=True)
            image.save(image_path)
            del item['image']
            del item['prop_label']
            del item['hate_fine_grained_label']
            f.write(json.dumps(item, ensure_ascii=False) + '\n')


## 2. Checking Data Splits Balance

Analyze the distribution of hate/non-hate labels across train, dev, and test splits:

In [ ]:
import json

def count_labels(file_path):
    hate = 0
    non_hate = 0
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            sample = json.loads(line)
            label = sample.get("hate_label")
            if label == 1:
                hate += 1
            elif label == 0:
                non_hate += 1
    return hate, non_hate

# Replace these with your actual file paths
files = {
    "train": "Prop2Hate-Meme/arabic_hateful_meme_train.jsonl",
    "dev": "Prop2Hate-Meme/arabic_hateful_meme_dev.jsonl",
    "test": "Prop2Hate-Meme/arabic_hateful_meme_test.jsonl"
}

for split, path in files.items():
    hate, non_hate = count_labels(path)
    total = hate + non_hate
    print(f"{split.upper()} — Total: {total}, Hate: {hate}, Non-Hate: {non_hate}, Hate %: {hate / total:.2%}")


## 3. Test on Gold Test Split

Load trained models and evaluate on the test set:


In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import torch
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# # Load gold labels and predictions
gold = pd.read_csv("task3_test_gold.csv")            # columns: id,label
pred1 = pd.read_csv("../scripts/prediction0_7333.csv") 

# align by id
df = gold.merge(pred1, on="id")
print("Check",df.head())
# get lists
y_true = df["testing_label"].tolist()
y_pred = df["prediction"].tolist()

# pick metrics
print("Accuracy :", accuracy_score(y_true, y_pred))
print("Precision:", precision_score(y_true, y_pred, pos_label="hate"))
print("Recall   :", recall_score(y_true, y_pred, pos_label="hate"))
print("F1       :", f1_score(y_true, y_pred, pos_label="hate"))
print("Macro F1 :", f1_score(y_true, y_pred, average="macro"))


## Results Visualization

In [ ]:
# Confusion Matrix
def plot_confusion_matrix(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(6, 4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['Non-hate', 'Hate'], 
                yticklabels=['Non-hate', 'Hate'])
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()

# plot_confusion_matrix(y_true, y_pred)